In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=31aca76d94a4251fdde58a96ac610f84301747e918ac90244644ef174582d0d8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

In [ ]:
df = spark.read.format("csv").load("sample_data/california_housing_train.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [ ]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
+---------+--------+----

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn('id', monotonically_increasing_id())

df = df[['id'] + df.columns[:-1]]

df.show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  0|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  1|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  2|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  3|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  4|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           6

In [ ]:
df.count()

17000

In [ ]:
from pyspark.sql.functions import mean, variance , col, count

# Calculate mean and variance for each selected column
mean_variance_df = df.select(mean('total_bedrooms'))

# Show the result
mean_variance_df.show()

+-------------------+
|avg(total_bedrooms)|
+-------------------+
|  539.4108235294118|
+-------------------+



In [ ]:
# Calculate mean for each column
mean_exprs = [mean(col(col_name)).alias('mean_' + col_name) for col_name in df.columns]

# Calculate variance for each column
variance_exprs = [variance(col(col_name)).alias('variance_' + col_name) for col_name in df.columns]

# Select the mean and variance for each column
mean_variance_df = df.select(mean_exprs + variance_exprs)

# Show the result
mean_variance_df.show()

+-------+-------------------+----------------+-----------------------+-----------------+-------------------+------------------+-----------------+------------------+-----------------------+-----------+------------------+-----------------+---------------------------+--------------------+-----------------------+-------------------+-------------------+----------------------+---------------------------+
|mean_id|     mean_longitude|   mean_latitude|mean_housing_median_age| mean_total_rooms|mean_total_bedrooms|   mean_population|  mean_households|mean_median_income|mean_median_house_value|variance_id|variance_longitude|variance_latitude|variance_housing_median_age|variance_total_rooms|variance_total_bedrooms|variance_population|variance_households|variance_median_income|variance_median_house_value|
+-------+-------------------+----------------+-----------------------+-----------------+-------------------+------------------+-----------------+------------------+-----------------------+--------

In [ ]:
df.select(*[mean(c) for c in df.columns]).show()

+-------+-------------------+----------------+-----------------------+-----------------+-------------------+------------------+-----------------+------------------+-----------------------+
|avg(id)|     avg(longitude)|   avg(latitude)|avg(housing_median_age)| avg(total_rooms)|avg(total_bedrooms)|   avg(population)|  avg(households)|avg(median_income)|avg(median_house_value)|
+-------+-------------------+----------------+-----------------------+-----------------+-------------------+------------------+-----------------+------------------+-----------------------+
| 8499.5|-119.56210823529375|35.6252247058827|      28.58935294117647|2643.664411764706|  539.4108235294118|1429.5739411764705|501.2219411764706| 3.883578100000021|     207300.91235294117|
+-------+-------------------+----------------+-----------------------+-----------------+-------------------+------------------+-----------------+------------------+-----------------------+



In [ ]:
# Check for null values in the "total_rooms" column
null_total_rooms_count = df.filter(col("total_rooms").isNull()).count()

# Show the result
print("Null values in 'total_rooms' column:", null_total_rooms_count)

Null values in 'total_rooms' column: 0


In [ ]:
# Count null values for each column
null_counts = [(col_name, df.filter(col(col_name).isNull()).count()) for col_name in df.columns]

# Show the result
for col_name, null_count in null_counts:
    print(f"Null values in '{col_name}' column: {null_count}")

Null values in 'id' column: 0
Null values in 'longitude' column: 0
Null values in 'latitude' column: 0
Null values in 'housing_median_age' column: 0
Null values in 'total_rooms' column: 0
Null values in 'total_bedrooms' column: 0
Null values in 'population' column: 0
Null values in 'households' column: 0
Null values in 'median_income' column: 0
Null values in 'median_house_value' column: 0


In [ ]:
train, test = df.randomSplit([0.7, 0.3])

train, test

(DataFrame[id: bigint, longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double],
 DataFrame[id: bigint, longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double])

In [ ]:
print('No of rows in train dataframe:: ',train.count())
print('No of columns in train dataframe:: ',len(train.columns))

No of rows in train dataframe::  11969
No of columns in train dataframe::  10


In [ ]:
numerical_features_lst = train.columns
numerical_features_lst.remove('median_house_value')
numerical_features_lst.remove('id')

numerical_features_lst

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [ ]:
# imputing missing values

from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=numerical_features_lst,
                  outputCols=numerical_features_lst)

imputer = imputer.fit(train)

train = imputer.transform(train)
test = imputer.transform(test)

train.show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  0|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  1|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  2|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  4|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  7|  -114.59|   34.83|              41.0|      812.0|         168.0|     375.0|     158.0|       1.7083|           4

In [ ]:
# the VectorAssembler class from PySpark's pyspark.ml.feature module to assemble multiple columns into a single vector column

from pyspark.ml.feature import VectorAssembler

if 'numerical_feature_vector' not in train.columns:
    numerical_vector_assembler = VectorAssembler(inputCols=numerical_features_lst,
                                                 outputCol='numerical_feature_vector')
    train = numerical_vector_assembler.transform(train)
else:
    print("'numerical_feature_vector' column already exists in train  DataFrame.")



if 'numerical_feature_vector' not in test.columns:
    numerical_vector_assembler = VectorAssembler(inputCols=numerical_features_lst,
                                                 outputCol='numerical_feature_vector')
    test = numerical_vector_assembler.transform(test)
else:
    print("'numerical_feature_vector' column already exists in test DataFrame.")

# Show the first 2 rows of the DataFrame
train.show(2)
test.show(2)

'numerical_feature_vector' column already exists in train  DataFrame.
'numerical_feature_vector' column already exists in test DataFrame.
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|numerical_feature_vector|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+
|  0|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|    [-114.31,34.19,15...|
|  1|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|    [-114.47,34.4,19....|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-

In [ ]:
train.select('numerical_feature_vector').take(2)

[Row(numerical_feature_vector=DenseVector([-114.31, 34.19, 15.0, 5612.0, 1283.0, 1015.0, 472.0, 1.4936])),
 Row(numerical_feature_vector=DenseVector([-114.47, 34.4, 19.0, 7650.0, 1901.0, 1129.0, 463.0, 1.82]))]

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='numerical_feature_vector',
                        outputCol='scaled_numerical_feature_vector',
                        withStd=True, withMean=True)

scaler = scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

train.show(3)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+-------------------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|numerical_feature_vector|scaled_numerical_feature_vector|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+-------------------------------+
|  0|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|    [-114.31,34.19,15...|           [2.62721873379178...|
|  1|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|    [-114.47,34.4,19....|           [2.54717726614910...|
|  2|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|    

In [ ]:
train.select('scaled_numerical_feature_vector').take(3)

[Row(scaled_numerical_feature_vector=DenseVector([2.6272, -0.6737, -1.0817, 1.355, 1.7488, -0.3574, -0.077, -1.2608])),
 Row(scaled_numerical_feature_vector=DenseVector([2.5472, -0.5752, -0.7624, 2.2868, 3.2045, -0.2593, -0.1003, -1.0883])),
 Row(scaled_numerical_feature_vector=DenseVector([2.5022, -0.9085, -0.922, -0.8817, -0.8635, -0.9443, -0.9971, -1.1777]))]

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='scaled_numerical_feature_vector',
                      labelCol='median_house_value')

lr

LinearRegression_c6bf0cc52245

In [ ]:
lr = lr.fit(train)

lr

LinearRegressionModel: uid=LinearRegression_c6bf0cc52245, numFeatures=8

In [ ]:
pred_train_df = lr.transform(train).withColumnRenamed('prediction',
                                                      'predicted_median_house_value')

pred_train_df.show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+-------------------------------+----------------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|numerical_feature_vector|scaled_numerical_feature_vector|predicted_median_house_value|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+-------------------------------+----------------------------+
|  0|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|    [-114.31,34.19,15...|           [2.62721873379178...|          -63.22529605982709|
|  1|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|    [-114.47,34.4,19....|  

In [ ]:
pred_test_df = lr.transform(test).withColumnRenamed('prediction', 'predicted_median_house_value')

pred_test_df.show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+-------------------------------+----------------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|numerical_feature_vector|scaled_numerical_feature_vector|predicted_median_house_value|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+-------------------------------+----------------------------+
|  3|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|    [-114.57,33.64,14...|           [2.49715134887243...|           36413.66592963578|
|  5|  -114.58|   33.63|              29.0|     1387.0|         236.0|     671.0|     239.0|       3.3438|           74000.0|    [-114.58,33.63,29...|  

In [ ]:
pred_test_pd_df = pred_test_df.toPandas()

pred_test_pd_df.head(2)

,id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,numerical_feature_vector,scaled_numerical_feature_vector,predicted_median_house_value
0,3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,"[-114.57, 33.64, 14.0, 1501.0, 337.0, 515.0, 2...","[2.497151348872434, -0.9319567155382452, -1.16...",36413.665930
1,5,-114.58,33.63,29.0,1387.0,236.0,671.0,239.0,3.3438,74000.0,"[-114.58, 33.63, 29.0, 1387.0, 236.0, 671.0, 2...","[2.492148757144764, -0.9366514023413658, 0.035...",45436.435886


In [ ]:
predictions_and_actuals = pred_test_df[['predicted_median_house_value',
                                        'median_house_value']]

predictions_and_actuals.show(5)

+----------------------------+------------------+
|predicted_median_house_value|median_house_value|
+----------------------------+------------------+
|           36413.66592963578|           73400.0|
|           45436.43588636213|           74000.0|
|           27428.73686657037|           82400.0|
|           30700.59244296592|           58400.0|
|         -27006.919978391117|           48600.0|
+----------------------------+------------------+
only showing top 5 rows



In [ ]:
predictions_and_actuals_rdd = predictions_and_actuals.rdd

predictions_and_actuals_rdd.take(2)

[Row(predicted_median_house_value=36413.66592963578, median_house_value=73400.0),
 Row(predicted_median_house_value=45436.43588636213, median_house_value=74000.0)]

In [ ]:
from pyspark.mllib.evaluation import RegressionMetrics

metrics = RegressionMetrics(predictions_and_actuals_rdd)

s = '''
Mean Squared Error:      {0}
Root Mean Squared Error: {1}
Mean Absolute Error:     {2}
R**2:                    {3}
'''.format(metrics.meanSquaredError,
           metrics.rootMeanSquaredError,
           metrics.meanAbsoluteError,
           metrics.r2
           )

print(s)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(



Mean Squared Error:      4687778656.291053
Root Mean Squared Error: 68467.35467572157
Mean Absolute Error:     50508.82636363511
R**2:                    0.6548871685293796

